In [4]:
import torch
import torch.nn as nn
torch.cuda.is_available()

True

In [5]:
class CA(nn.Module):
    def __init__(self, state_dim=16, hidden_dim=128):
        super(CA, self).__init__()
        self.state_dim = state_dim

dupa
